### Installations

In [1]:
!pip install holidays

### Libraries

In [2]:
import pandas as pd
import calendar
from datetime import date, timedelta
import matplotlib.pyplot as plt
import argparse

try:
    import holidays
except ImportError:
    raise ImportError("Please install the 'holidays' library: pip install holidays")

### Load Templates

In [3]:
dfFixtures_template = pd.read_csv('Input/fixtureTable.csv', dayfirst=True, parse_dates=['date'])  # unshifted
dfTeams_base = pd.read_csv('Input/teamTable.csv')            # no attributes
dfCompetitions = pd.read_csv('Input/competitionTable.csv')

### Helper Functions for Date Shift

In [4]:
import pandas as pd
import calendar
from datetime import date
from typing import Optional
import holidays

# -----------------------------------------------------------------------------
# 1. Initialize UK Holiday Engine (Dynamic Year Support)
# -----------------------------------------------------------------------------
def get_uk_holiday_name(dt: date) -> Optional[str]:
    return holidays.UK(years=dt.year).get(dt)

def find_holiday_date_by_name(target_year: int, holiday_name: str) -> Optional[date]:
    """
    Look up the date of a holiday in the target year by its name.
    """
    target_holidays = holidays.UK(years=target_year)
    for dt, name in target_holidays.items():
        if name == holiday_name:
            return dt
    return None


# -----------------------------------------------------------------------------
# 2. Weekday Ordinal Preservation Logic
# -----------------------------------------------------------------------------
def get_ordinal_weekday(dt: date) -> int:
    """Return the 1-based occurrence of dt.weekday() in dt.month."""
    month_cal = calendar.monthcalendar(dt.year, dt.month)
    weekday = dt.weekday()
    occurrences = [week[weekday] for week in month_cal if week[weekday] != 0]
    for i, day in enumerate(occurrences, start=1):
        if day == dt.day:
            return i
    raise ValueError(f"Could not determine ordinal for date {dt}")


def find_nth_weekday(year: int, month: int, weekday: int, ordinal: int) -> date:
    """Find the date of the ordinal-th weekday in the given month/year."""
    month_cal = calendar.monthcalendar(year, month)
    occurrences = [week[weekday] for week in month_cal if week[weekday] != 0]
    if not occurrences:
        raise ValueError(f"No occurrences of weekday {weekday} in {month}/{year}")
    if ordinal <= len(occurrences):
        day = occurrences[ordinal - 1]
    else:
        day = occurrences[-1]
    return date(year, month, day)


# -----------------------------------------------------------------------------
# 3. Shift Logic with Holiday + Weekday Ordinal Fallback
# -----------------------------------------------------------------------------
def shift_fixture_date(original_date: date, target_year: int) -> date:
    """
    Shift a single date into target_year, preserving bank holidays and weekday ordinals.
    """
    orig_holiday = get_uk_holiday_name(original_date)
    if orig_holiday:
        holiday_match = find_holiday_date_by_name(target_year, orig_holiday)
        if holiday_match:
            return holiday_match

    ordinal = get_ordinal_weekday(original_date)
    return find_nth_weekday(
        target_year,
        original_date.month,
        original_date.weekday(),
        ordinal
    )


# -----------------------------------------------------------------------------
# 4. Rebase Fixture Dates
# -----------------------------------------------------------------------------
def rebase_fixtures(df: pd.DataFrame, target_start_year: int) -> pd.DataFrame:
    """
    Rebase fixture dates in `df` to a new season starting in `target_start_year`.
    """
    df_new = df.copy()
    if 'round' in df_new.columns:
        df_new.drop(columns=['round'], inplace=True)

    df_new['date'] = pd.to_datetime(df_new['date'], errors='coerce')
    original_start_year = int(df_new['date'].dt.year.min())

    def _shift(ts):
        if pd.isna(ts):
            return pd.NaT
        orig = ts.date()
        year_offset = orig.year - original_start_year
        new_year = target_start_year + year_offset
        return shift_fixture_date(orig, new_year)

    df_new['date'] = df_new['date'].apply(_shift)
    df_new['date'] = df_new['date'].apply(
        lambda d: d.strftime('%Y-%m-%d') if pd.notna(d) else pd.NaT
    )

    return df_new

# -----------------------------------------------------------------------------
# 5. Entry Point: Fixture Creation for a Season
# -----------------------------------------------------------------------------
def create_fixture_list(year: int) -> pd.DataFrame:
    """Create rebased fixture list for a new season."""
    return rebase_fixtures(dfFixtures_template.copy(), year)


### Helper Functions for Team Attributes

In [5]:
import pandas as pd
import random

def position_to_attribute(pos: int, min_val: float, max_val: float, min_rank: int, max_rank: int, ascending=True) -> float:
    """
    Linearly scale attribute from min_val to max_val over [min_rank, max_rank].
    If ascending=False, scales from max_val to min_val instead.
    """
    pos = max(min_rank, min(pos, max_rank))
    scale = (pos - min_rank) / (max_rank - min_rank) if max_rank > min_rank else 0
    if not ascending:
        scale = 1.0 - scale
    return round(min_val + (max_val - min_val) * scale, 2)


def assign_team_attributes(df: pd.DataFrame, seed: int = None) -> pd.DataFrame:
    """
    Assigns 'defense', 'midfield', and 'attack' attributes based on 'lastPos'.
    """
    rng = random.Random(seed)
    df_out = df.copy()

    def compute_attributes(row):
        pos = row['lastPos']
        if 1 <= pos <= 116:
            base = position_to_attribute(pos, min_val=1.0, max_val=8.5, min_rank=1, max_rank=116, ascending=False)
            variation = 0.75
        elif 117 <= pos <= 120:
            base = position_to_attribute(pos, min_val=6.0, max_val=8.0, min_rank=117, max_rank=120, ascending=True)
            variation = 0.5
        elif 121 <= pos <= 124:
            base = position_to_attribute(pos, min_val=7.0, max_val=9.0, min_rank=121, max_rank=124, ascending=True)
            variation = 0.5
        else:
            base = 5.0
            variation = 0.5

        return {
            'defense': round(base + rng.uniform(-variation, variation), 2),
            'midfield': round(base + rng.uniform(-variation, variation), 2),
            'attack': round(base + rng.uniform(-variation, variation), 2)
        }

    attributes = df_out.apply(compute_attributes, axis=1, result_type='expand')
    df_out[['defense', 'midfield', 'attack']] = attributes
    return df_out

def generate_team_attributes() -> pd.DataFrame:
    """Generate team attributes from base lastPos table."""
    return assign_team_attributes(dfTeams_base.copy())


### Match Engine

In [6]:
import pandas as pd
import random
from typing import Tuple

def rnd_num() -> float:
    """Return a random number between 0 and 1."""
    return random.uniform(0, 1)

def goal_chance() -> int:
    """40% chance to score a goal."""
    return 1 if rnd_num() > 0.6 else 0

def get_team_attributes(team_id: int, df: pd.DataFrame) -> list:
    """Get defense, midfield, attack as a list for given team ID."""
    row = df.loc[df['teamID'] == team_id]
    if row.empty:
        raise ValueError(f"Team ID {team_id} not found in dataframe.")
    return [row.iloc[0]['defense'], row.iloc[0]['midfield'], row.iloc[0]['attack']]

def play_match_half(team1: list, team2: list) -> list:
    """
    Simulates one half of a football match between two teams.
    Each team is a list of 3 attributes: [def, mid, att]
    """
    match_seq = list(range(1, 13))
    random.shuffle(match_seq)

    result = [0, 0]  # [home_goals, away_goals]

    for index in range(1, 11):
        if match_seq[0] == index and team1[2] > team2[0] and (random.random() * team1[2]) > (random.random() * team2[0]):
            result[0] += 1 if random.random() > 0.6 else 0
        elif match_seq[6] == index and (random.random() * team1[2]) > (random.random() * team2[0]) and (random.random() * team1[1]) > (random.random() * team2[1]):
            result[0] += 1 if random.random() > 0.6 else 0
        elif match_seq[1] == index and team2[2] > team1[0] and (random.random() * team2[2]) > (random.random() * team1[0]):
            result[1] += 1 if random.random() > 0.6 else 0
        elif match_seq[7] == index and (random.random() * team1[2]) > (random.random() * team1[0]) and (random.random() * team2[1]) > (random.random() * team1[1]):
            result[1] += 1 if random.random() > 0.6 else 0
        elif match_seq[2] == index and team1[1] > team2[1] and (random.random() * team1[1]) > (random.random() * team2[1]):
            result[0] += 1 if random.random() > 0.6 else 0
        elif match_seq[8] == index and (random.random() * team1[1]) > (random.random() * team2[1]):
            result[0] += 1 if random.random() > 0.6 else 0
        elif match_seq[3] == index and team2[1] > team1[1] and (random.random() * team2[1]) > (random.random() * team1[1]):
            result[1] += 1 if random.random() > 0.6 else 0
        elif match_seq[9] == index and (random.random() * team2[1]) > (random.random() * team1[1]):
            result[1] += 1 if random.random() > 0.6 else 0
        elif match_seq[4] == index and random.random() > 0.45:
            result[0] += 1 if random.random() > 0.6 else 0

    return result

def play_full_match(team1_id: int, team2_id: int, df: pd.DataFrame) -> list:
    """
    Play a full match between two teams and return the result [goals_team1, goals_team2].
    """
    team1 = get_team_attributes(team1_id, df)
    team2 = get_team_attributes(team2_id, df)

    first_half = play_match_half(team1, team2)
    second_half = play_match_half(team1, team2)

    final_result = [first_half[0] + second_half[0], first_half[1] + second_half[1]]
    return final_result

import random

def simulate_penalties(home_team_id: int, away_team_id: int, dfTeams) -> tuple:
    """
    Run a simple 5‐kick penalty shootout for each team, then sudden death if still tied.
    Returns: (home_pk_goals, away_pk_goals, winner_id).
    """
    # Get [defense, midfield, attack] for each side
    attrs_home = get_team_attributes(home_team_id, dfTeams)  # → list [def, mid, att]
    attrs_away = get_team_attributes(away_team_id, dfTeams)

    # Use 'attack' at index 2, and 'defense' at index 0
    home_att = attrs_home[2]
    home_def = attrs_home[0]
    away_att = attrs_away[2]
    away_def = attrs_away[0]

    # Base penalty conversion probability: blend of home_att vs away_def
    base_prob_home = 0.8 * (home_att / (home_att + away_def)) + 0.2
    base_prob_away = 0.8 * (away_att / (away_att + home_def)) + 0.2

    # Clamp between [0.5, 0.95]
    base_prob_home = min(max(base_prob_home, 0.5), 0.95)
    base_prob_away = min(max(base_prob_away, 0.5), 0.95)

    def take_five_kicks(p_success: float) -> int:
        """Simulate exactly 5 penalty kicks given success probability p_success."""
        made = 0
        for _ in range(5):
            if random.random() < p_success:
                made += 1
        return made

    # 1) First 5 kicks each
    home_score = take_five_kicks(base_prob_home)
    away_score = take_five_kicks(base_prob_away)

    # 2) Sudden death if still tied
    while home_score == away_score:
        if random.random() < base_prob_home:
            home_score += 1
        if random.random() < base_prob_away:
            away_score += 1
        # repeat until one is ahead

    winner_id = home_team_id if home_score > away_score else away_team_id
    return home_score, away_score, winner_id

def play_cup_match(home_team_id: int, away_team_id: int, dfTeams):
    """
    Simulate a single cup tie:
      1) Play 90′ via play_full_match(...)
      2) If draw, run one “extra‐time half” via play_match_half(...)
      3) If still draw, run penalties via simulate_penalties(...)
    Returns a dict with:
      - home_goals (FT + ET)
      - away_goals (FT + ET)
      - winner_id
      - extra_time_played (bool)
      - home_et_goals, away_et_goals (ints; zero if no ET)
      - home_pk, away_pk (ints; None if no shootout)
    """
    # 1) Full‐time (90′)
    home_ft, away_ft = play_full_match(home_team_id, away_team_id, dfTeams)

    # If someone wins in 90′, skip ET/PK
    if home_ft != away_ft:
        winner = home_team_id if home_ft > away_ft else away_team_id
        return {
            "home_goals": home_ft,
            "away_goals": away_ft,
            "winner_id":  winner,
            "extra_time_played": False,
            "home_et_goals": 0,
            "away_et_goals": 0,
            "home_pk":      None,
            "away_pk":      None
        }

    # 2) Extra‐time: simulate one “half” of 45′ via play_match_half(...)
    # (This effectively approximates a 30′ ET; you can adjust scaling later if needed.)
    et_home, et_away = play_match_half(
        get_team_attributes(home_team_id, dfTeams),
        get_team_attributes(away_team_id, dfTeams)
    )
    home_total = home_ft + et_home
    away_total = away_ft + et_away

    # If someone wins in ET, skip penalties
    if home_total != away_total:
        winner = home_team_id if home_total > away_total else away_team_id
        return {
            "home_goals": home_total,
            "away_goals": away_total,
            "winner_id":  winner,
            "extra_time_played": True,
            "home_et_goals":  et_home,
            "away_et_goals":  et_away,
            "home_pk":       None,
            "away_pk":       None
        }

    # 3) Penalty shootout
    home_pk, away_pk, winner = simulate_penalties(home_team_id, away_team_id, dfTeams)
    return {
        "home_goals": home_total,
        "away_goals": away_total,
        "winner_id":  winner,
        "extra_time_played": True,
        "home_et_goals":  et_home,
        "away_et_goals":  et_away,
        "home_pk":       home_pk,
        "away_pk":       away_pk
    }

### Helper Function to Initialise Cup Status

In [7]:
import random

def initialize_cup_status(dfTeams):
    """
    Reset and set initial cup‐eligibility flags based on last season’s results:
      - inCS: Community Shield (league winner & FA Cup winner)
      - inCL: Champions League (top‐4 English, CL winner override; plus direct qualifiers lastPos ∈ {121–124})
      - inEL: Europa League (4 English slots: FA & LC winners + next best by position; 
                             plus direct qualifiers lastPos ∈ {117–120})
      - inFA: FA Cup R1 (teamID 45–92 direct; + 32 random from teamID 93–134)
      - inLC: League Cup (teamID 21–92)
    """
    # 0) Ensure lastPos is integer
    dfTeams['lastPos'] = dfTeams['lastPos'].astype(int)

    # 1) Reset all flags
    for col in ['inCS','inCL','inEL','inFA','inLC']:
        dfTeams[col] = False

    # 2) Community Shield: English league winner & FA Cup winner
    cs_mask = (
        (dfTeams['nationality']=='ENG') &
        ((dfTeams['lastPos']==1) | (dfTeams['faCupWin']==True))
    )
    dfTeams.loc[cs_mask, 'inCS'] = True

    # 3) Champions League (English slots)
    eng = dfTeams['nationality']=='ENG'
    cl_winner = dfTeams[eng & (dfTeams['clCupWin']==True)]
    if not cl_winner.empty:
        wid = cl_winner['teamID'].iat[0]
        dfTeams.loc[dfTeams['teamID']==wid, 'inCL'] = True
        others = dfTeams[eng & (dfTeams['teamID']!=wid)].nsmallest(3, 'lastPos')
        dfTeams.loc[others.index, 'inCL'] = True
    else:
        top4 = dfTeams[eng].nsmallest(4, 'lastPos')
        dfTeams.loc[top4.index, 'inCL'] = True
    # 3b) Direct CL qualifiers by lastPos codes 121–124
    dfTeams.loc[dfTeams['lastPos'].isin([121,122,123,124]), 'inCL'] = True

    # 4) Europa League (English slots)
    el_winners = dfTeams[
        eng &
        ((dfTeams['faCupWin']==True) | (dfTeams['lcCupWin']==True)) &
        (~dfTeams['inCL'])
    ]['teamID'].tolist()
    dfTeams.loc[dfTeams['teamID'].isin(el_winners), 'inEL'] = True

    # 4b) Top-up English EL slots to 4 by best lastPos among ENG not inCL/EL
    curr = dfTeams[(eng) & (dfTeams['inEL']==True)].shape[0]
    if curr < 4:
        needed = 4 - curr
        fill = dfTeams[
            eng & (~dfTeams['inCL']) & (~dfTeams['inEL'])
        ].nsmallest(needed, 'lastPos')
        dfTeams.loc[fill.index, 'inEL'] = True

    # 4c) Direct EL qualifiers by lastPos codes 117–120
    dfTeams.loc[dfTeams['lastPos'].isin([117,118,119,120]), 'inEL'] = True

    # 5) FA Cup R1: teamID 45–92 direct + random 32 from teamID 93–134
    direct_ids = dfTeams.loc[dfTeams['teamID'].between(45, 92), 'teamID'].tolist()
    pool_ids   = dfTeams.loc[dfTeams['teamID'].between(93, 134), 'teamID'].tolist()
    if len(pool_ids) < 32:
        raise ValueError(f"Not enough teams in 93–134 to pick 32; found {len(pool_ids)}")
    fa_qual = random.sample(pool_ids, 32)
    dfTeams.loc[dfTeams['teamID'].isin(direct_ids + fa_qual), 'inFA'] = True

    # 6) League Cup: teamID 21–92
    dfTeams.loc[dfTeams['teamID'].between(21, 92), 'inLC'] = True

    # 7) Summary
    print("✅ Initialized cup status:")
    print(f"  inCS: {dfTeams['inCS'].sum()} teams")
    print(f"  inCL: {dfTeams['inCL'].sum()} teams")
    print(f"  inEL: {dfTeams['inEL'].sum()} teams")
    print(f"  inFA: {dfTeams['inFA'].sum()} teams")
    print(f"  inLC: {dfTeams['inLC'].sum()} teams")

    return dfTeams


### Helper Functions for League Round

In [8]:
def play_league_round(day: int,
                      dfFixtures: pd.DataFrame,
                      dfTeams: pd.DataFrame,
                      dfCompetitions: pd.DataFrame,
                      dfGroupStage: pd.DataFrame = None) -> pd.DataFrame:
    """
    Plays all league‐style fixtures (domestic + UCL/EL group‐stage) for a given dayCounter,
    updates dfFixtures in‐place, prints results by competition, then:
      1) Recompute domestic tables (comps 1–5)
      2) If dfGroupStage is provided and EL (8) matches were played, recompute EL tables
      3) If dfGroupStage is provided and UCL (9) matches were played, recompute UCL tables
    """

    # 1) Build a “not played” mask treating NaN as False
    not_played_mask = ~dfFixtures['played'].fillna(False)

    # 2) Include domestic (1–5), EL (8), UCL (9)
    valid_comp_ids = [1, 2, 3, 4, 5, 8, 9]

    day_fixtures = dfFixtures[
        (dfFixtures['dayCounter'] == day) &
        (not_played_mask) &
        (dfFixtures['competitionID'].isin(valid_comp_ids))
    ].copy()

    if day_fixtures.empty:
        print(f"No league/group‐stage fixtures scheduled or all already played for day {day}.")
        return dfFixtures

    # 3) Build lookup maps
    team_name_map = dfTeams.set_index('teamID')['teamName'].to_dict()
    comp_name_map = dfCompetitions.set_index('competitionID')['competitionName'].to_dict()

    results_by_comp = {}
    match_date = day_fixtures.iloc[0]['date']

    # Track which competitions had a match today
    comps_played_today = set()

    for idx, match in day_fixtures.iterrows():
        home_id = match['homeTeam']
        away_id = match['awayTeam']
        comp_id = match['competitionID']

        # Simulate 90' only
        home_goals, away_goals = play_full_match(home_id, away_id, dfTeams)

        # Update dfFixtures
        dfFixtures.at[idx, 'homeGoals'] = home_goals
        dfFixtures.at[idx, 'awayGoals'] = away_goals
        dfFixtures.at[idx, 'played'] = True

        comps_played_today.add(comp_id)

        # Format for printing
        home_name = team_name_map.get(home_id, f"Team{home_id}")
        away_name = team_name_map.get(away_id, f"Team{away_id}")
        comp_name = comp_name_map.get(comp_id, f"Competition{comp_id}")
        line = f"{home_name} {home_goals} - {away_goals} {away_name}"
        results_by_comp.setdefault(comp_id, []).append(line)

    # 4) Print results sorted by competitionID
    print(f"\nMatches played on {match_date}:")
    for comp_id in sorted(results_by_comp.keys()):
        comp_name = comp_name_map.get(comp_id, f"Competition{comp_id}")
        print(comp_name)
        for line in results_by_comp[comp_id]:
            print(line)

    # 5) Recompute domestic league tables (1–5)
    update_tables()

    # 6) If dfGroupStage is provided, update EL/UCL group tables as needed
    if dfGroupStage is not None:
        # Europa League (comp 8)
        if 8 in comps_played_today:
            global el_tables
            el_tables = generate_euro_group_tables(
                competitionID=8,
                dfFixtures=dfFixtures,
                dfTeams=dfTeams,
                dfGroupStage=dfGroupStage
            )
         
        # Champions League (comp 9)
        if 9 in comps_played_today:
            global ucl_tables
            ucl_tables = generate_euro_group_tables(
                competitionID=9,
                dfFixtures=dfFixtures,
                dfTeams=dfTeams,
                dfGroupStage=dfGroupStage
            )
          
    return dfFixtures

### Helper Functions for League Tables

In [9]:
def generate_league_tables(dfFixtures: pd.DataFrame, dfTeams: pd.DataFrame) -> dict:
    """
    Generate league tables for CompetitionIDs 1 to 5, ensuring all teams are included even if they haven't played.
    Removes teamID and formats GF, GA, GD as integers.

    Returns:
        Dictionary {competitionID: league_table_df}
    """
    league_tables = {}

    for comp_id in range(1, 6):
        # Get all teams in this competition (home and away)
        teams_in_league = pd.unique(
            dfFixtures[dfFixtures['competitionID'] == comp_id][['homeTeam', 'awayTeam']].values.ravel()
        )

        # Initialize stats
        team_stats = {
            team_id: {'MP': 0, 'W': 0, 'D': 0, 'L': 0, 'GF': 0, 'GA': 0, 'Points': 0}
            for team_id in teams_in_league
        }

        # Played matches only
        league_matches = dfFixtures[
            (dfFixtures['competitionID'] == comp_id) & (dfFixtures['played'])
        ]

        # Accumulate stats
        for _, match in league_matches.iterrows():
            h_id, a_id = match['homeTeam'], match['awayTeam']
            hg, ag = match['homeGoals'], match['awayGoals']

            # Home update
            team_stats[h_id]['MP'] += 1
            team_stats[h_id]['GF'] += hg
            team_stats[h_id]['GA'] += ag
            # Away update
            team_stats[a_id]['MP'] += 1
            team_stats[a_id]['GF'] += ag
            team_stats[a_id]['GA'] += hg

            if hg > ag:
                team_stats[h_id]['W'] += 1
                team_stats[h_id]['Points'] += 3
                team_stats[a_id]['L'] += 1
            elif hg == ag:
                team_stats[h_id]['D'] += 1
                team_stats[a_id]['D'] += 1
                team_stats[h_id]['Points'] += 1
                team_stats[a_id]['Points'] += 1
            else:
                team_stats[a_id]['W'] += 1
                team_stats[a_id]['Points'] += 3
                team_stats[h_id]['L'] += 1

        # Build rows
        rows = []
        for team_id in teams_in_league:
            stats = team_stats[team_id]
            team_name = dfTeams.loc[dfTeams['teamID'] == team_id, 'teamName'].values[0]
            gd = stats['GF'] - stats['GA']
            rows.append({
                'teamName': team_name,
                'MP': stats['MP'],
                'W': stats['W'],
                'D': stats['D'],
                'L': stats['L'],
                'GF': int(round(stats['GF'])),
                'GA': int(round(stats['GA'])),
                'GD': int(round(gd)),
                'Points': stats['Points']
            })

        league_df = pd.DataFrame(rows)
        league_df = league_df.sort_values(by=['Points', 'GD', 'GF'], ascending=False).reset_index(drop=True)
        league_df.insert(0, 'Position', league_df.index + 1)

        league_tables[comp_id] = league_df

    return league_tables


def print_all_league_tables(league_tables: dict, dfCompetitions: pd.DataFrame):
    """
    Pretty print all league tables with their competition names as headers.

    Parameters:
        league_tables: Dict {competitionID: league DataFrame}
        dfCompetitions: DataFrame containing competitionID and competitionName
    """
    # Build a mapping from competitionID to name
    comp_name_map = dfCompetitions.set_index('competitionID')['competitionName'].to_dict()

    for comp_id in sorted(league_tables.keys()):
        table = league_tables[comp_id]
        comp_name = comp_name_map.get(comp_id, f"Competition {comp_id}")

        print(f"\n{comp_name}")
        print(table.to_string(index=False))


### Helper Functions for Cup Rounds

In [10]:
def play_cup_round(dayCounter, competitionID, dfFixtures, dfTeams):
    """
    Simulate every unplayed cup fixture on each day in 'dayCounter' for competitionID,
    writing the following into dfFixtures for each match:
      - homeGoals, awayGoals: total goals after 90' + ET (if any)
      - homeGoalsAET, awayGoalsAET: extra‐time goals
      - homePens, awayPens: penalty goals (None if no shootout)
      - played: True once done
    Prints a summary including “(a.e.t.)” or “[p x–y]” as needed.
    """
    # 1) Look up competition name (uses global dfCompetitions)
    comp_row = dfCompetitions.loc[
        dfCompetitions['competitionID'] == competitionID,
        'competitionName'
    ]
    comp_name = comp_row.values[0] if not comp_row.empty else f"ID {competitionID}"

    # 2) Parse dayCounter into a list of ints
    if isinstance(dayCounter, str):
        days = [int(x.strip()) for x in dayCounter.split(',') if x.strip().isdigit()]
    elif isinstance(dayCounter, (list, tuple)):
        days = [int(x) for x in dayCounter]
    else:
        days = [int(dayCounter)]

    all_winners = []
    for day in days:
        # 3) Filter for unplayed cup fixtures on this day & competition
        mask = (
            (dfFixtures['dayCounter']   == day) &
            (dfFixtures['competitionID'] == competitionID) &
            (dfFixtures['played']        == False)
        )
        todays_matches = dfFixtures.loc[mask].copy()
        if todays_matches.empty:
            print(f"No fixtures to play on day {day} for '{comp_name}'")
            continue

        for idx, match in todays_matches.iterrows():
            # skip any blank placeholder
            if pd.isna(match['homeTeam']) or pd.isna(match['awayTeam']):
                print(f"❌ Skipping empty slot on day {day} for '{comp_name}' at index {idx}")
                continue

            home_id = int(match['homeTeam'])
            away_id = int(match['awayTeam'])

            # Simulate 90′ + ET + penalties
            result = play_cup_match(home_id, away_id, dfTeams)
            all_winners.append(result['winner_id'])

            # 4) Write results back into dfFixtures
            dfFixtures.at[idx, 'homeGoals']     = result['home_goals']
            dfFixtures.at[idx, 'awayGoals']     = result['away_goals']
            dfFixtures.at[idx, 'homeGoalsAET']  = result['home_et_goals']
            dfFixtures.at[idx, 'awayGoalsAET']  = result['away_et_goals']
            dfFixtures.at[idx, 'homePens']      = result['home_pk']
            dfFixtures.at[idx, 'awayPens']      = result['away_pk']
            dfFixtures.at[idx, 'played']        = True

            # 5) Print summary line
            home_name   = dfTeams.loc[dfTeams['teamID']==home_id, 'teamName'].iat[0]
            away_name   = dfTeams.loc[dfTeams['teamID']==away_id, 'teamName'].iat[0]
            winner_name = dfTeams.loc[dfTeams['teamID']==result['winner_id'], 'teamName'].iat[0]

            suffix = ""
            if result['extra_time_played']:
                suffix += " (a.e.t.)"
            if result['home_pk'] is not None and result['away_pk'] is not None:
                suffix += f" [p {result['home_pk']}–{result['away_pk']}]"

            print(
                f"Day {day} | {comp_name} | "
                f"{home_name} {result['home_goals']}–{result['away_goals']} {away_name}"
                f"{suffix} → Winner: {winner_name}"
            )

    return all_winners


### Generic Cup Draw

In [11]:
import random

def draw_cup(row, dfTeams, dfFixtures, dfCompetitions):
    """
    Generic cup‐draw function for:
      • Community Shield   (competitionID=10, flag 'inCS', exactly 2 teams)
      • League Cup         (competitionID=6,  flag 'inLC', even-numbered field)
      • FA Cup Round 1     (competitionID=7,  flag 'inFA', even-numbered field)
      • Europa League KO   (competitionID=8,  flag 'inEL', even-numbered field)
      • Champions League KO(competitionID=9,  flag 'inCL', even-numbered field)

    Assumes dfTeams['inXX'] flags have already been set for that competition.
    1) Read comp_id, look up its flag column.
    2) Select all dfTeams where that flag is True.
       • If comp_id == 10, require exactly 2.
       • Otherwise require an even number.
    3) Shuffle and pair off in order.
    4) Parse allowed_days from row['dayCounter'] (comma-sep ints).
    5) In dfFixtures, find blank fixtures for this comp on those days:
       competitionID == comp_id, dayCounter ∈ allowed_days,
       homeTeam/isnull & awayTeam/isnull.
    6) Fill the first N placeholders with your pairs.
    7) Print a summary.
    """
    comp_id = int(row['competitionID'])
    comp_name = dfCompetitions.loc[
        dfCompetitions['competitionID'] == comp_id, 'competitionName'
    ].iat[0]

    # 1) Hard-coded mapping of competition → which inXX column to use
    comp_flag_map = {
        10: 'inCS',
        6: 'inLC',
        7: 'inFA',
        8: 'inEL',
        9: 'inCL'
    }
    if comp_id not in comp_flag_map:
        print(f"❌ No cup-draw configured for competitionID {comp_id}")
        return
    flag_col = comp_flag_map[comp_id]

    # 2) Gather eligible teams
    eligible = dfTeams[dfTeams[flag_col] == True].copy()
    n = len(eligible)
    if comp_id == 10:
        if n != 2:
            print(f"❌ Error: Community Shield needs 2 teams, found {n}")
            return
    else:
        if n % 2 != 0:
            print(f"❌ Error: {comp_name} needs an even number of teams, found {n}")
            return

    # 3) Shuffle and pair
    team_ids = eligible['teamID'].tolist()
    random.shuffle(team_ids)
    pairs = [(team_ids[i], team_ids[i+1]) for i in range(0, len(team_ids), 2)]

    # 4) Parse allowed_days
    allowed_days = [
        int(d.strip()) for d in str(row['dayCounter']).split(',')
        if d.strip().isdigit()
    ]
    if not allowed_days:
        print(f"❌ Error: No valid dayCounters provided for {comp_name}")
        return

    # 5) Find blank fixtures
    mask = (
        (dfFixtures['competitionID'] == comp_id) &
        (dfFixtures['dayCounter'].isin(allowed_days)) &
        (dfFixtures['homeTeam'].isnull()) &
        (dfFixtures['awayTeam'].isnull())
    )
    open_fx = dfFixtures.loc[mask]
    if len(open_fx) < len(pairs):
        print(f"❌ Error: Not enough blank fixtures for {comp_name}: "
              f"needed {len(pairs)}, found {len(open_fx)}")
        return

    # 6) Fill placeholders
    for (home_tid, away_tid), idx in zip(pairs, open_fx.index):
        dfFixtures.at[idx, 'homeTeam'] = home_tid
        dfFixtures.at[idx, 'awayTeam'] = away_tid

    # 7) Print summary
    dates = dfFixtures.loc[open_fx.index[:len(pairs)], 'date'].unique().tolist()
    print(f"\n🏆 {comp_name} draw ({len(pairs)} matches) across: {', '.join(dates)}")
    for i, (home_tid, away_tid) in enumerate(pairs, 1):
        home_name = dfTeams.loc[dfTeams['teamID'] == home_tid, 'teamName'].iat[0]
        away_name = dfTeams.loc[dfTeams['teamID'] == away_tid, 'teamName'].iat[0]
        dayc = dfFixtures.at[open_fx.index[i-1], 'dayCounter']
        print(f"  Match {i:02d} (day {dayc}): {home_name} vs {away_name}")

    # commit back
    globals()['dfFixtures'] = dfFixtures


### European Helper Functions

In [12]:
import random
import pandas as pd

def draw_euro_group_stage(row, dfTeams, dfFixtures, dfCompetitions):
    """
    Generic draw for European group stages (UCL or EL), but now appends to a global dfGroupStage
    instead of overwriting it. This means you can call it twice (once for competitionID=9, once
    for competitionID=8) and keep both sets of group assignments.

    Expects:
      - row['competitionID'] ∈ {9 (UCL), 8 (EL)}
      - row['dayCounter']  = comma-separated string of 12 ints (for UCL) or 6 ints (for EL)
      - dfTeams, dfFixtures, dfCompetitions already in scope

    After this runs, global dfGroupStage will contain rows:
      competitionID | teamID | groupID
    for all previously‐drawn competitions (8 and/or 9), plus this new draw.
    """

    # 1) Parse dayCounters
    raw = row['dayCounter']
    if isinstance(raw, str):
        date_list = [int(x.strip()) for x in raw.split(',') if x.strip().isdigit()]
    else:
        try:
            date_list = list(raw)
        except:
            date_list = [int(raw)]

    comp_id = int(row['competitionID'])
    comp_row = dfCompetitions.loc[
        dfCompetitions['competitionID'] == comp_id, 'competitionName'
    ]
    comp_name = comp_row.values[0] if not comp_row.empty else f"Competition{comp_id}"

    # 2) Determine flag column and expected number of dayCounters
    if comp_id == 9:    # UCL
        expected = 12
        flag_col = 'inCL'
    elif comp_id == 8:  # EL
        expected = 6
        flag_col = 'inEL'
    else:
        raise ValueError(f"CompetitionID {comp_id} is not UCL (9) or EL (8).")

    if len(date_list) != expected:
        raise ValueError(f"{comp_name} requires {expected} dayCounters, got {len(date_list)}.")

    # 3) Select exactly 32 teams still in the competition
    euro_teams = dfTeams.loc[dfTeams[flag_col] == True, ['teamID', 'nationality']].to_dict(orient='records')
    if len(euro_teams) != 32:
        raise ValueError(f"Expected 32 teams where {flag_col}==True, found {len(euro_teams)}.")

    # 4) Assign to 8 groups of 4 with unique nationality constraint
    def attempt_grouping():
        random.shuffle(euro_teams)
        groups = {gid: [] for gid in range(1, 9)}
        for team in euro_teams:
            placed = False
            order = list(groups.keys())
            random.shuffle(order)
            for gid in order:
                if len(groups[gid]) < 4:
                    existing_nats = {t['nationality'] for t in groups[gid]}
                    if team['nationality'] not in existing_nats:
                        groups[gid].append(team)
                        placed = True
                        break
            if not placed:
                return None
        return groups

    groups = None
    for _ in range(1000):
        groups = attempt_grouping()
        if groups is not None:
            break
    if groups is None:
        raise RuntimeError("Failed to assign teams into nationality‐unique groups after many tries.")

    # 5) Build a DataFrame for this draw (comp_id, teamID, groupID)
    new_group_rows = [
        {'competitionID': comp_id, 'teamID': t['teamID'], 'groupID': gid}
        for gid, members in groups.items()
        for t in members
    ]
    new_df = pd.DataFrame(new_group_rows)

    # 6) Append to global dfGroupStage (dropping any old rows for the same comp_id)
    global dfGroupStage
    try:
        # If dfGroupStage already exists, remove any rows for this comp_id, then append
        dfGroupStage = dfGroupStage[ dfGroupStage['competitionID'] != comp_id ].copy()
        dfGroupStage = pd.concat([dfGroupStage, new_df], ignore_index=True)
    except NameError:
        # dfGroupStage doesn't exist yet; just create it
        dfGroupStage = new_df.copy()

    # 7) Fill placeholders in dfFixtures for matchups (same as before)
    def single_round_robin(team_ids):
        teams = list(team_ids)
        n = len(teams)  # 4
        rounds = []
        for _ in range(n - 1):  # 3 rounds
            pairings = []
            for j in range(n // 2):
                t1 = teams[j]
                t2 = teams[n - 1 - j]
                pairings.append((t1, t2))
            rounds.append(pairings)
            teams = [teams[0]] + [teams[-1]] + teams[1:-1]
        return rounds

    group_schedules = {}
    for gid, members in groups.items():
        tids = [t['teamID'] for t in members]
        first = single_round_robin(tids)
        second = [[(away, home) for (home, away) in rnd] for rnd in first]
        full = first + second  # 6 total rounds
        group_schedules[gid] = full

    # 8) Populate existing dfFixtures placeholders:
    for rnd_idx in range(6):
        if comp_id == 9:
            # UCL: two dayCounters per round
            dc_1_4 = date_list[2 * rnd_idx]
            dc_5_8 = date_list[2 * rnd_idx + 1]
        else:
            # EL: single dayCounter per round
            dc_all = date_list[rnd_idx]

        for gid, schedule in group_schedules.items():
            match_pairs = schedule[rnd_idx]  # two (home,away) pairs
            dc = (dc_1_4 if gid <= 4 else dc_5_8) if comp_id == 9 else dc_all

            placeholder_mask = (
                (dfFixtures['competitionID'] == comp_id) &
                (dfFixtures['dayCounter']     == dc) &
                (dfFixtures['homeTeam'].isna()) &
                (dfFixtures['awayTeam'].isna())
            )
            placeholders = dfFixtures.loc[placeholder_mask]
            needed = len(match_pairs)
            if len(placeholders) < needed:
                raise RuntimeError(
                    f"Not enough placeholder rows for {comp_name} on day {dc}: "
                    f"needed {needed}, found {len(placeholders)}"
                )

            indices = placeholders.index.tolist()[:needed]
            for idx, (home_tid, away_tid) in zip(indices, match_pairs):
                dfFixtures.at[idx, 'homeTeam'] = home_tid
                dfFixtures.at[idx, 'awayTeam'] = away_tid
                dfFixtures.at[idx, 'round']    = rnd_idx + 1

    # 9) Print out group composition (for just this competition)
    print(f"\nPopulated {comp_name} Group Stage (competitionID={comp_id}):")
    for gid, members in groups.items():
        names = [
            dfTeams.loc[dfTeams['teamID'] == t['teamID'], 'teamName'].values[0]
            for t in members
        ]
        print(f"  Group {gid}: {', '.join(names)}")

    return dfGroupStage

import pandas as pd

def generate_euro_group_tables(competitionID: int,
                                dfFixtures: pd.DataFrame,
                                dfTeams: pd.DataFrame,
                                dfGroupStage: pd.DataFrame) -> dict:
    """
    Returns a dict mapping groupID → DataFrame of standings for that group,
    for the given European group-stage competition (8 = EL or 9 = UCL).

    Each DataFrame has columns:
      ['Position', 'Team', 'Played', 'Wins', 'Draws', 'Losses',
       'GoalsFor', 'GoalsAgainst', 'GoalDiff', 'Points']

    Args:
      competitionID: 8 or 9
      dfFixtures:    DataFrame with group-stage matches, including:
                      ['competitionID','homeTeam','awayTeam','homeGoals','awayGoals','played']
      dfTeams:       DataFrame with ['teamID','teamName', …]
      dfGroupStage:  DataFrame with ['competitionID','teamID','groupID']

    Returns:
      A dict { groupID: group_table_df, … } for groupID in [1..8].
    """

    # 1) Filter to played matches of this competition
    mask = (
        (dfFixtures['competitionID'] == competitionID) &
        (dfFixtures['played'] == True)
    )
    played = dfFixtures.loc[mask].copy()
    if played.empty:
        print(f"No played fixtures found for competition {competitionID}.")
        return {}

    # 2) Merge in groupID for home and away teams
    #    We assume dfGroupStage has exactly one row per (competitionID, teamID).
    gs = dfGroupStage[dfGroupStage['competitionID'] == competitionID][['teamID','groupID']]

    played = played.merge(gs.rename(columns={'teamID':'homeTeam', 'groupID':'homeGroup'}),
                          on='homeTeam', how='left')
    played = played.merge(gs.rename(columns={'teamID':'awayTeam', 'groupID':'awayGroup'}),
                          on='awayTeam', how='left')

    # 3) Build a “per‐team‐per‐match” record
    records = []
    for _, row in played.iterrows():
        h_id = int(row['homeTeam'])
        a_id = int(row['awayTeam'])
        hg   = int(row['homeGoals'])
        ag   = int(row['awayGoals'])
        hg_id = row['homeGroup']
        ag_id = row['awayGroup']
        # (They should be equal: hg_id == ag_id, otherwise data is inconsistent.)

        # Home team stats
        if hg > ag:
            h_w, h_d, h_l, h_pts = 1, 0, 0, 3
            a_w, a_d, a_l, a_pts = 0, 0, 1, 0
        elif hg < ag:
            h_w, h_d, h_l, h_pts = 0, 0, 1, 0
            a_w, a_d, a_l, a_pts = 1, 0, 0, 3
        else:
            h_w, h_d, h_l, h_pts = 0, 1, 0, 1
            a_w, a_d, a_l, a_pts = 0, 1, 0, 1

        records.append({
            'groupID':      hg_id,
            'teamID':       h_id,
            'Played':       1,
            'Wins':         h_w,
            'Draws':        h_d,
            'Losses':       h_l,
            'GoalsFor':     hg,
            'GoalsAgainst': ag,
            'Points':       h_pts
        })
        # Away team stats
        records.append({
            'groupID':      ag_id,
            'teamID':       a_id,
            'Played':       1,
            'Wins':         a_w,
            'Draws':        a_d,
            'Losses':       a_l,
            'GoalsFor':     ag,
            'GoalsAgainst': hg,
            'Points':       a_pts
        })

    df_stats = pd.DataFrame(records)

    # 4) Aggregate by groupID + teamID
    agg = df_stats.groupby(['groupID','teamID'], as_index=False).sum()
    #    Compute goal difference
    agg['GoalDiff'] = agg['GoalsFor'] - agg['GoalsAgainst']

    # 5) Merge in team names
    agg = agg.merge(dfTeams[['teamID','teamName']], on='teamID', how='left')

    # 6) For each group, create a sorted table
    group_tables = {}
    for gid in sorted(agg['groupID'].unique()):
        gdf = agg[agg['groupID'] == gid].copy()
        # Sort by Points desc, then GoalDiff desc, then GoalsFor desc
        gdf = gdf.sort_values(
            by=['Points','GoalDiff','GoalsFor'],
            ascending=[False, False, False]
        ).reset_index(drop=True)

        # Assign position 1–4
        gdf.insert(0, 'Position', range(1, len(gdf)+1))

        # Reorder columns for readability
        gdf = gdf[['Position','teamName','Played','Wins','Draws','Losses',
                   'GoalsFor','GoalsAgainst','GoalDiff','Points']]
        gdf.rename(columns={'teamName':'Team'}, inplace=True)

        group_tables[gid] = gdf

    return group_tables

import pandas as pd

import pandas as pd

def print_euro_group_tables(comp_id: int,
                            dfTeams: pd.DataFrame,
                            dfFixtures: pd.DataFrame,
                            dfGroupStage: pd.DataFrame):
    """
    Prints all 8 group standings for the given European competition (8=EL or 9=UCL).
    - If no matches have been played in a group, that group prints all zeros.
    - If some matches have been played, you'll see real stats (Played/W/D/L/GF/GA/GD/Pts).

    Args:
      comp_id       : 8 for Europa League, 9 for Champions League
      dfTeams       : DataFrame with ['teamID','teamName', …]
      dfFixtures    : DataFrame with all group fixtures, including 'played', 'homeGoals', 'awayGoals'
      dfGroupStage  : DataFrame with ['competitionID','teamID','groupID'] (32 rows for comp_id)

    Usage:
      # Right after drawing groups (so dfGroupStage has comp_id rows):
      print_euro_group_tables(9, dfTeams, dfFixtures, dfGroupStage)
      
      # After some matchdays have been simulated:
      print_euro_group_tables(9, dfTeams, dfFixtures, dfGroupStage)
    """

    # 1) Build mapping: groupID → [teamID, …] for this competition
    gs = dfGroupStage[dfGroupStage['competitionID'] == comp_id]
    if len(gs) != 32:
        raise ValueError(
            f"dfGroupStage must contain exactly 32 rows for comp {comp_id}, "
            f"found {len(gs)}"
        )

    euro_groups = gs.groupby('groupID')['teamID'].apply(list).to_dict()
    missing = [gid for gid in range(1, 9) if gid not in euro_groups]
    if missing:
        raise ValueError(f"Missing groupIDs {missing} in dfGroupStage for comp {comp_id}.")

    # 2) Generate actual group tables (this will return a dict {gid: DataFrame} for groups
    #    where at least one match has been played; empty dict if none)
    actual_tables = generate_euro_group_tables(
        competitionID=comp_id,
        dfFixtures=dfFixtures,
        dfTeams=dfTeams,
        dfGroupStage=dfGroupStage
    )

    # 3) For each group (1..8), either print the actual table or a zero‐stats template
    for gid in range(1, 9):
        comp_name = dfCompetitions.loc[
            dfCompetitions['competitionID'] == comp_id, 'competitionName'
        ].values[0]
        print(f"\n--- {comp_name} Group {gid} Standings ---")

        if gid in actual_tables:
            tbl = actual_tables[gid].copy()
            # Ensure there’s a Position column in front
            if 'Position' not in tbl.columns:
                tbl.insert(0, 'Position', range(1, len(tbl) + 1))
            print(tbl.to_string(index=False))
            continue

        # No matches played yet in this group: build zero‐stats
        rows = []
        for tid in euro_groups[gid]:
            team_name = dfTeams.loc[dfTeams['teamID'] == tid, 'teamName'].values[0]
            rows.append({
                'Team':         team_name,
                'Played':       0,
                'Wins':         0,
                'Draws':        0,
                'Losses':       0,
                'GoalsFor':     0,
                'GoalsAgainst': 0,
                'GoalDiff':     0,
                'Points':       0
            })
        zero_df = pd.DataFrame(rows)
        zero_df.insert(0, 'Position', range(1, 5))
        print(zero_df.to_string(index=False))

import random

import random

def end_cl_groups(row, dfTeams, dfFixtures, dfGroupStage):
    """
    After UCL group stage completes:
      1) Reset all dfTeams['inCL'] to False.
      2) Re-flag the 16 teams (8 group winners + 8 runners-up).
      3) Draw two-legged Round of 16:
         - first half of row['dayCounter'] for leg1 days,
         - second half for leg2 days,
         cycling if fewer days than matches,
         and populate existing blanks (one fixture slot per match per leg).
    """
    comp_id = 9

    # 1) Clear previous CL flags
    dfTeams['inCL'] = False

    # 2) Find winners & runners-up
    tables = generate_euro_group_tables(
        competitionID=comp_id,
        dfFixtures=dfFixtures,
        dfTeams=dfTeams,
        dfGroupStage=dfGroupStage
    )
    winners = []
    runners  = []
    for gid, tbl in tables.items():
        # map names back to IDs
        win_name, run_name = tbl.iloc[0]['Team'], tbl.iloc[1]['Team']
        win_id = dfTeams.loc[dfTeams.teamName == win_name, 'teamID'].iat[0]
        run_id = dfTeams.loc[dfTeams.teamName == run_name, 'teamID'].iat[0]
        winners.append((gid, win_id))
        runners.append((gid, run_id))

    # flag them
    dfTeams.loc[dfTeams.teamID.isin([tid for _,tid in winners] + [tid for _,tid in runners]), 'inCL'] = True

    # 3) Pair winners vs runners from different groups
    rng = list(range(len(winners)))  # should be 8
    for _ in range(1000):
        random.shuffle(runners)
        if all(winners[i][0] != runners[i][0] for i in rng):
            break
    else:
        raise RuntimeError("Cannot pair winners with runners without rematches")

    match_pairs = [(winners[i][1], runners[i][1]) for i in rng]
    n_matches = len(match_pairs)

    # 4) Parse dayCounters and split into two pools
    days = [int(x.strip()) for x in str(row['dayCounter']).split(',') if x.strip().isdigit()]
    if not days:
        raise ValueError("No valid dayCounters provided for End CL Groups")
    half = len(days) // 2 or 1
    leg1_pool = days[:half]
    leg2_pool = days[half:half*2] if len(days) > half else days

    # build full schedules, cycling
    leg1_days = [leg1_pool[i % len(leg1_pool)] for i in range(n_matches)]
    leg2_days = [leg2_pool[i % len(leg2_pool)] for i in range(n_matches)]

    # 5) Fill placeholders for leg1 then leg2
    # leg1
    for (home_tid, away_tid), dc in zip(match_pairs, leg1_days):
        mask = (
            (dfFixtures.competitionID == comp_id) &
            (dfFixtures.dayCounter    == dc) &
            dfFixtures.homeTeam.isnull() &
            dfFixtures.awayTeam.isnull()
        )
        idxs = dfFixtures.loc[mask].index
        if idxs.empty:
            raise ValueError(f"No blank fixture for leg1 match on day {dc}")
        dfFixtures.at[idxs[0], 'homeTeam'] = home_tid
        dfFixtures.at[idxs[0], 'awayTeam'] = away_tid

    # leg2 (reverse)
    for (home_tid, away_tid), dc in zip(match_pairs, leg2_days):
        mask = (
            (dfFixtures.competitionID == comp_id) &
            (dfFixtures.dayCounter    == dc) &
            dfFixtures.homeTeam.isnull() &
            dfFixtures.awayTeam.isnull()
        )
        idxs = dfFixtures.loc[mask].index
        if idxs.empty:
            raise ValueError(f"No blank fixture for leg2 match on day {dc}")
        # reverse home/away
        dfFixtures.at[idxs[0], 'homeTeam'] = away_tid
        dfFixtures.at[idxs[0], 'awayTeam'] = home_tid

    # 6) Summary
    print(f"\n🏆 UCL Round of 16 (two-legged) drawn:")
    for i, ((h, a), d1, d2) in enumerate(zip(match_pairs, leg1_days, leg2_days), 1):
        h_name = dfTeams.loc[dfTeams.teamID==h, 'teamName'].iat[0]
        a_name = dfTeams.loc[dfTeams.teamID==a, 'teamName'].iat[0]
        print(f"  Match {i:02d}: Leg1 (day {d1}) {h_name} vs {a_name}; "
              f"Leg2 (day {d2}) {a_name} vs {h_name}")

    # commit back
    globals()['dfFixtures'] = dfFixtures

import random

def end_el_groups(row, dfTeams, dfFixtures, dfGroupStage):
    """
    After EL group stage completes:
      1) Reset dfTeams['inEL'] to False.
      2) Re-flag 24 teams: 8 EL group winners, 8 EL runners-up, and 8 CL third-place finishers.
      3) Draw and populate two-legged “Knockout play-offs” between each EL runner-up
         and one CL third-place, avoiding same-nationality clashes.
    """
    el_id = 8
    cl_id = 9
    # 1) clear inEL
    dfTeams['inEL'] = False

    # 2a) EL group winners & runners-up
    el_tables = generate_euro_group_tables(
        competitionID=el_id,
        dfFixtures=dfFixtures,
        dfTeams=dfTeams,
        dfGroupStage=dfGroupStage
    )
    winners = []
    runners  = []
    for gid, tbl in el_tables.items():
        win_name = tbl.iloc[0]['Team']
        run_name = tbl.iloc[1]['Team']
        win_id = dfTeams.loc[dfTeams.teamName==win_name,'teamID'].iat[0]
        run_id = dfTeams.loc[dfTeams.teamName==run_name,'teamID'].iat[0]
        winners.append(win_id)
        runners.append(run_id)

    # 2b) CL third-place finishers
    cl_tables = generate_euro_group_tables(
        competitionID=cl_id,
        dfFixtures=dfFixtures,
        dfTeams=dfTeams,
        dfGroupStage=dfGroupStage
    )
    thirds = []
    for gid, tbl in cl_tables.items():
        third_name = tbl.iloc[2]['Team']
        third_id   = dfTeams.loc[dfTeams.teamName==third_name,'teamID'].iat[0]
        thirds.append(third_id)

    # re-flag all 24 inEL
    dfTeams.loc[dfTeams.teamID.isin(winners + runners + thirds), 'inEL'] = True

    # 3) Pair each EL runner-up with a CL third-place, no same-nationality
    def same_nation(e, c):
        return dfTeams.loc[dfTeams.teamID==e, 'nationality'].iat[0] == \
               dfTeams.loc[dfTeams.teamID==c, 'nationality'].iat[0]

    paired = False
    for _ in range(1000):
        random.shuffle(thirds)
        if all(not same_nation(runners[i], thirds[i]) for i in range(8)):
            paired = True
            break
    if not paired:
        raise RuntimeError("Cannot pair without same-nation clash")

    match_pairs = list(zip(runners, thirds))  # 8 pairs

    # 4) Parse dayCounters
    days = [int(d) for d in str(row['dayCounter']).split(',') if d.strip().isdigit()]
    if not days:
        raise ValueError("No dayCounters provided for EL playoffs")
    half = len(days)//2 or 1
    leg1_pool = days[:half]
    leg2_pool = days[half:half*2] if len(days)>half else days

    # cycle through pools
    leg1_days = [leg1_pool[i % len(leg1_pool)] for i in range(8)]
    leg2_days = [leg2_pool[i % len(leg2_pool)] for i in range(8)]

    # 5) Fill placeholders for leg1 and leg2
    # leg1: runner-up home
    for (run_id, third_id), dc in zip(match_pairs, leg1_days):
        mask = (
            (dfFixtures.competitionID==el_id) &
            (dfFixtures.dayCounter   ==dc) &
            dfFixtures.homeTeam.isnull() &
            dfFixtures.awayTeam.isnull()
        )
        idxs = dfFixtures.loc[mask].index
        if idxs.empty:
            raise ValueError(f"No blank EL playoff fixture for leg1 on day {dc}")
        dfFixtures.at[idxs[0], 'homeTeam'] = run_id
        dfFixtures.at[idxs[0], 'awayTeam'] = third_id

    # leg2: third-place home
    for (run_id, third_id), dc in zip(match_pairs, leg2_days):
        mask = (
            (dfFixtures.competitionID==el_id) &
            (dfFixtures.dayCounter   ==dc) &
            dfFixtures.homeTeam.isnull() &
            dfFixtures.awayTeam.isnull()
        )
        idxs = dfFixtures.loc[mask].index
        if idxs.empty:
            raise ValueError(f"No blank EL playoff fixture for leg2 on day {dc}")
        dfFixtures.at[idxs[0], 'homeTeam'] = third_id
        dfFixtures.at[idxs[0], 'awayTeam'] = run_id

    # 6) Print summary
    comp_name = dfCompetitions.loc[dfCompetitions.competitionID==el_id,'competitionName'].iat[0]
    print(f"\n🏆 {comp_name} Knockout Play-offs (two-legged):")
    for i, ((r, c), d1, d2) in enumerate(zip(match_pairs, leg1_days, leg2_days), 1):
        r_name = dfTeams.loc[dfTeams.teamID==r,'teamName'].iat[0]
        c_name = dfTeams.loc[dfTeams.teamID==c,'teamName'].iat[0]
        print(f"  Match {i:02d}: Leg1 (day {d1}) {r_name} vs {c_name}; "
              f"Leg2 (day {d2}) {c_name} vs {r_name}")

    globals()['dfFixtures'] = dfFixtures



### The Flow

In [13]:
# Load verbose flow plan
dfSeasonFlow = pd.read_csv('Input/theFlow.csv')

# Initialise flow state only once
try:
    flow_state
except NameError:
    flow_state = {'step': 0}  # this advances by 1 each time the cell is run


In [14]:
# 🔄 Reset simulation flow to the start
flow_state = {'step': 0}
print("✅ Flow state reset. You can now re-run the simulation from the beginning.")

✅ Flow state reset. You can now re-run the simulation from the beginning.


### The Flow Helper Functions

In [15]:
def play_league_day(day):
    """
    Wrapper around play_league_round. If dfGroupStage is not defined yet,
    we simply pass None and skip European table updates inside play_league_round.
    """
    global dfFixtures, dfGroupStage  # dfGroupStage may or may not exist

    try:
        # If dfGroupStage exists, pass it in
        dfFixtures = play_league_round(
            day=day,
            dfFixtures=dfFixtures,
            dfTeams=dfTeams,
            dfCompetitions=dfCompetitions,
            dfGroupStage=dfGroupStage
        )
    except NameError:
        # dfGroupStage doesn’t exist yet—pass None instead
        dfFixtures = play_league_round(
            day=day,
            dfFixtures=dfFixtures,
            dfTeams=dfTeams,
            dfCompetitions=dfCompetitions,
            dfGroupStage=None
        )

def update_tables():
    global league_tables
    league_tables = generate_league_tables(dfFixtures, dfTeams)

def print_tables():
    print_all_league_tables(league_tables, dfCompetitions)
    
def create_new_fixture_list():
    global dfFixtures
    dfFixtures = create_fixture_list(season_state['current_year'])
    print(f"✅ Fixture list created for {season_state['current_year']}")

def create_team_attributes():
    global dfTeams
    dfTeams = generate_team_attributes()
    print(f"✅ Team attributes created for {season_state['current_year']}")

def update_cup_status(dayCounter, competitionID, dfFixtures, dfTeams):
    """
    For each day in 'dayCounter' (which may be an int or a comma-separated string of ints),
    find all played fixtures for that competition on those days, determine the losers
    (taking into account ET and penalties), and set their corresponding 'inX' flag to False.
    
    Mapping of competitionID → dfTeams flag:
      6  → 'inLC'
      7  → 'inFA'
      8  → 'inEL'
      9  → 'inCL'
      10 → 'inCS'
    """
    # 1) Parse dayCounter into a list of ints
    if isinstance(dayCounter, str):
        day_list = [int(d.strip()) for d in dayCounter.split(',') if d.strip().isdigit()]
    else:
        # assume it's already an int (or something convertible)
        day_list = [int(dayCounter)]
    
    # 2) Map competitionID to the dfTeams boolean column
    comp_flag_map = {
        6:  'inLC',
        7:  'inFA',
        8:  'inEL',
        9:  'inCL',
        10: 'inCS'
    }
    if competitionID not in comp_flag_map:
        print(f"[Warning] No flag mapping for competitionID {competitionID}")
        return
    
    flag_col = comp_flag_map[competitionID]

    # 3) Iterate over each specified day
    for day in day_list:
        # Look up competition name (optional; for nicer printing)
        comp_row = dfCompetitions.loc[
            dfCompetitions['competitionID'] == competitionID,
            'competitionName'
        ]
        comp_name = comp_row.values[0] if not comp_row.empty else f"ID {competitionID}"

        # Filter for fixtures matching this day & competition, and that have been played
        mask = (
            (dfFixtures['dayCounter']   == day) &
            (dfFixtures['competitionID'] == competitionID) &
            (dfFixtures['played']        == True)
        )
        played_matches = dfFixtures.loc[mask].copy()
        if played_matches.empty:
            print(f"No played fixtures found on day {day} for '{comp_name}'")
            continue

        # 4) For each match, determine loser and update dfTeams
        for idx, match in played_matches.iterrows():
            home_id    = int(match['homeTeam'])
            away_id    = int(match['awayTeam'])
            home_goals = int(match['homeGoals'])
            away_goals = int(match['awayGoals'])

            # Determine loser (accounting for ET/penalties if needed)
            if home_goals > away_goals:
                loser_id = away_id
            elif away_goals > home_goals:
                loser_id = home_id
            else:
                # tied after ET, so look at penalty columns
                home_pens = match.get('homePens')
                away_pens = match.get('awayPens')
                if home_pens is None or away_pens is None:
                    print(f"[Error] Tie on day {day}, comp '{comp_name}', but no penalty data.")
                    continue
                loser_id = home_id if home_pens < away_pens else away_id

            # Set that team’s 'inX' flag to False
            dfTeams.loc[dfTeams['teamID'] == loser_id, flag_col] = False

            # Print which team was knocked out
            loser_name = dfTeams.loc[
                dfTeams['teamID'] == loser_id, 'teamName'
            ].values[0]
            print(
                f"Day {day} | {comp_name} | Knocked out: {loser_name} → {flag_col} = False"
            )

def add_lc_teams_r2(dfTeams):
    """
    Include all teams with teamID ≤ 20 into League Cup Round 2,
    except those already in Europa League or Champions League.
    Sets dfTeams['inLC'] = True for any teamID ≤ 20 where
    both inEL and inCL are False.
    """
    mask = (dfTeams['teamID'] <= 20) & (~dfTeams['inEL']) & (~dfTeams['inCL'])
    dfTeams.loc[mask, 'inLC'] = True
    added = dfTeams.loc[mask, 'teamName'].tolist()
    print(f"Added to LC Round 2: {', '.join(added)}")

def add_lc_teams_r3(dfTeams):
    """
    Include all teams with teamID ≤ 20 into League Cup Round 3
    if they are in Europa League (inEL) or Champions League (inCL).
    Sets dfTeams['inLC'] = True for any team meeting those criteria.
    """
    mask = (dfTeams['teamID'] <= 20) & (dfTeams['inEL'] | dfTeams['inCL'])
    dfTeams.loc[mask, 'inLC'] = True
    added = dfTeams.loc[mask, 'teamName'].tolist()
    print(f"Added to LC Round 3: {', '.join(added)}")
    
def add_fa_teams_r3(dfTeams):
    """
    Include all teams with teamID ≤ 20 into League Cup Round 3
    if they are in Europa League (inEL) or Champions League (inCL).
    Sets dfTeams['inLC'] = True for any team meeting those criteria.
    """
    mask = (dfTeams['teamID'] <= 44) 
    dfTeams.loc[mask, 'inFA'] = True
    added = dfTeams.loc[mask, 'teamName'].tolist()
    print(f"Added to FA Cup Round 3: {', '.join(added)}")
    
# Map flow text to actual logic
flow_dispatch = {
    'Create New Fixture List': lambda row: create_new_fixture_list(),
    'Create Team Attributes': lambda row: create_team_attributes(),
    'Play League Matches': lambda row: play_league_day(int(row['dayCounter'])),
    'Print Tables': lambda row: print_tables(),
    'Play Cup Round': lambda row: play_cup_round(row['dayCounter'], int(row['competitionID']), dfFixtures, dfTeams),
    'Update Cup Status': lambda row: update_cup_status(row['dayCounter'], int(row['competitionID']), dfFixtures, dfTeams),
    'Add LC Round2 Teams': lambda row: add_lc_teams_r2(dfTeams),
    'Add LC Round3 Teams': lambda row: add_lc_teams_r3(dfTeams),
    'Draw European Group Stage': lambda row: draw_euro_group_stage(row, dfTeams, dfFixtures, dfCompetitions),
    'Draw Cup' : lambda row: draw_cup(row, dfTeams, dfFixtures, dfCompetitions),
    'Initialize Cup Status' : lambda row: initialize_cup_status(dfTeams),
    'End CL Groups' : lambda row: end_cl_groups(row, dfTeams, dfFixtures, dfGroupStage),
    'End EL Groups' : lambda row: end_el_groups(row, dfTeams, dfFixtures, dfGroupStage),
    'Add FA Round3 Teams' : lambda row: add_fa_teams_r3(dfTeams)
}

In [16]:
# 📅 Initialize multi-season simulation state
try:
    season_state
except NameError:
    season_state = {
        'step': 0,
        'current_year': 2024,
        'fixtures_archive': {},
        'tables_archive': {}
    }
    print("✅ Season state initialized")


✅ Season state initialized


In [ ]:
# Step through the flow one step at a time
if flow_state['step'] < len(dfSeasonFlow):
    row = dfSeasonFlow.iloc[flow_state['step']]
    step_desc = row['functionToCall']
    print(f"\n=== STEP {flow_state['step']+1}: {step_desc} ===")

    # Run corresponding logic
    if step_desc in flow_dispatch:
        flow_dispatch[step_desc](row)
    else:
        print(f"[Skipped] No function mapped for: {step_desc}")

    flow_state['step'] += 1
else:
    print("=== All flow steps completed ===")

In [17]:
# Step through the flow in one hit to XX
def run_flow_to(target_step):
    """
    Advance the flow from its current position up to (but not including) target_step.
    target_step is 1-based: to run steps 1–80, call run_flow_to(80).
    """
    global flow_state, dfSeasonFlow, flow_dispatch

    # Convert target_step to zero-based index
    end_index = target_step

    while flow_state['step'] < len(dfSeasonFlow) and flow_state['step'] < end_index:
        row = dfSeasonFlow.iloc[flow_state['step']]
        step_desc = row['functionToCall']
        print(f"\n=== STEP {flow_state['step']+1}: {step_desc} ===")

        if step_desc in flow_dispatch:
            flow_dispatch[step_desc](row)
        else:
            print(f"[Skipped] No function mapped for: {step_desc}")

        flow_state['step'] += 1

    if flow_state['step'] >= end_index:
        print(f"\n=== Reached step {target_step}. Stopped. Current pointer: {flow_state['step']} ===")
    else:
        print("\n=== All flow steps completed before reaching the target ===")

run_flow_to(160)


=== STEP 1: Create New Fixture List ===
✅ Fixture list created for 2024

=== STEP 2: Create Team Attributes ===
✅ Team attributes created for 2024

=== STEP 3: Initialize Cup Status ===
✅ Initialized cup status:
  inCS: 2 teams
  inCL: 32 teams
  inEL: 32 teams
  inFA: 80 teams
  inLC: 72 teams

=== STEP 4: Draw Cup ===

🏆 Community Shield draw (1 matches) across: 2024-08-04
  Match 01 (day 3): Manchester City vs Arsenal

=== STEP 5: Draw Cup ===

🏆 League Cup draw (36 matches) across: 2024-08-13, 2024-08-14
  Match 01 (day 4): Leicester vs Morecambe
  Match 02 (day 4): Rotherham vs Derby
  Match 03 (day 4): Tranmere vs Sheffield Wednesday
  Match 04 (day 4): Gillingham vs Norwich
  Match 05 (day 4): Birmingham vs Blackburn
  Match 06 (day 4): Fleetwood Town vs Bristol City
  Match 07 (day 4): Walsall vs Sutton Utd
  Match 08 (day 4): Plymouth vs Notts County
  Match 09 (day 4): Stockport County vs Swindon Town
  Match 10 (day 4): Wycombe vs Port Vale
  Match 11 (day 4): Forest Green 

### Still in Cup

In [ ]:
eligible_teams = dfTeams[dfTeams['inFA'] == True].copy()
print(len(eligible_teams))
pd.set_option('display.max_rows', None)
eligible_teams

### Print League Tables

In [ ]:
print_all_league_tables(league_tables, dfCompetitions)

In [ ]:
print_euro_group_tables(8, dfTeams, dfFixtures, dfGroupStage)

In [ ]:
clgames = dfFixtures[dfFixtures['competitionID'] == 7].copy()

In [ ]:
pd.set_option('display.max_rows', None)
print(len(clgames))
clgames